<a href="https://colab.research.google.com/github/fullStackLucas/PROJETO_IMERSAO_ALURA/blob/main/Lucas_Gemini_Project_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Chatbot Mecânico com Google Generative AI**

Este projeto demonstra a construção de um chatbot especializado em diagnóstico de problemas automotivos, utilizando o Google Generative AI. O chatbot simula o conhecimento de um mecânico experiente e pode interagir com usuários para ajudá-los a identificar possíveis problemas em seus veículos.

#**Tecnologias Utilizadas**

*   **Google Generative AI:** Plataforma de IA que fornece modelos de linguagem poderosos para geração de texto.
*   **Python:** Linguagem de programação utilizada para interagir com a API do Google Generative AI.
*   **Google Colab:** Ambiente de notebook online que facilita a execução e o compartilhamento do código.

#**Como Funciona**

O chatbot utiliza um modelo de linguagem do Google Generative AI para processar as entradas do usuário e gerar respostas coerentes. Como o suporte direto para "instruções de sistema" não estava disponível até novembro de 2023, o projeto implementa um workaround:

*   **Prompt Elaborado:** Um prompt detalhado é usado para fornecer ao modelo informações sobre seu papel, objetivos, conhecimento e estilo de comunicação.
*   **Histórico do Chat:** Uma mensagem fictícia contendo as instruções é adicionada ao histórico do chat para simular instruções de sistema.


#**Executando o Projeto**



1.   **Configure o Ambiente:**

  *   Instale as bibliotecas necessárias (google-generativeai).
  *   Obtenha sua chave de API do Google Cloud Platform.


2.   **Execute o Código:**
  *   Importe as bibliotecas e configure a API.
  *   Defina o prompt e a mensagem fictícia com as instruções.
  *   Inicie o chat e interaja com o chatbot.


#**Limitações**


*   **Sem Suporte Nativo para Instruções de Sistema:** O projeto usa um workaround para simular instruções de sistema, o que pode ter limitações.
*   **Conhecimento Limitado:** O conhecimento do chatbot é baseado no modelo de linguagem utilizado e pode não ser totalmente preciso ou atualizado.


#**Possíveis Melhorias**



*   **Fine-tuning do Modelo:** Treinar o modelo com dados específicos sobre mecânica automotiva poderia melhorar sua precisão e conhecimento.
*   **Integração com Banco de Dados de Peças:** Conectar o chatbot a um banco de dados real de peças permitiria fornecer informações mais precisas sobre preços e fornecedores.
*   **Interface de Usuário:** Implementar uma interface de usuário mais amigável e interativa.



**INSTALAÇÃO DO SDK**

In [1]:
!pip install -q -U google-generativeai

**CONFIGURANDO O GENAI**

IMPORTANTE!

NA VARIÁVEL `system_instruction` ESTÃO AS DIRETRIZES QUE O MODELO DEVE SEGUIR.
NESSE CASO ESPECÍFICO ELE SERÁ UM MECÂNICO ALTAMENTE ESPECIALIZADO.


In [85]:
# Import the Python SDK
import google.generativeai as genai
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('api_key')
genai.configure(api_key=GOOGLE_API_KEY)

gen_config = {
    "temperature": 0.6,
    "candidate_count": 1,
}

safety_settings = {
    "HARASSMENT": "high",
    "HATE": "high",
    "SEXUAL": "high",
    "DANGEROUS": "high",
}

system_instruction = """
ESSA É APENAS UMA INSTRUÇÃO DE SISTEMA. RESPONDA À PRIMEIRA ITERAÇÃO SEMPRE
CORDIALMENTE E PERGUNTANDO EM QUE PODE AJUDAR!!
Você será um mecânico avançado, construído com o
conhecimento de milhares de mecânicos, engenheiros e especialistas automotivos
em todo o mundo. Sua função é fornecer um pré-diagnóstico com base na informação
o usuário lhe fornecer sobre a marca, modelo, ano, quilometragem e a descrição
do problema do seu veículo. Você deverá ter também acesso a uma vasta base de
dados de peças automobilísticas e pode fornecer sugestões de preços e
fornecedores confiáveis ao usuário.
  Sempre pergunte ao final de cada iteração se o problema do usuário foi resolvido.
  Caso o usuário dê uma resposta afirmativa, instrua-o a digitar: FIM
"""

LISTAGEM DE MODELOS QUE SUPORTAM O MÉTODO *generate_content*

In [82]:
for m in genai.list_models():
  if "generateContent" in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


**INICIALIZAÇÃO DO MODELO**

AQUI TAMBÉM TEMOS A INICIALIZAÇÃO DO CHATBOT

In [86]:
model = genai.GenerativeModel(
  model_name='gemini-1.5-pro-latest',
)

chat = model.start_chat(history=[])


**NESSE PONTO, VAMOS INICIALIZAR TODO O NOSSO FLUXO DE CONVERSAÇÃO**

A VARIÁVEL AUXILIAR `num_of_interactions` TEM O PAPEL DE LIMITAR O NUMERO DE VEZES QUE INTERAGIMOS COM O MODELO PARA A CONVERSA NÃO SE EXTENDER DEMAIS.

FIQUE À VONTADE PARA EXCLUÍ-LA DO CÓDIGO.
**NÃO ESQUEÇA DE EXCLUIR TAMBÉM OS LUGARES ONDE ELA É UTILIZADA!!**



In [ ]:
continue_chat = True
num_of_interactions = 0

initial_user_prompt = {
    "1": "Tenho um problema no meu carro, mas não tenho ideia do que seja. Poderia me ajudar?",
    "2": "Tenho um problema no meu carro, tenho uma ideia do que seja, mas ainda preciso de ajuda.",
    "3": "Tenho o diagnóstico de um mecânico, mas gostaria de uma segunda opinião",
    "4": "Meu carro está com problemas, já sei do que se trata, mas gostaria que você me indicasse onde comprar as peças que vou te fornecer. Sempre me pergunte antes as características do meu carro para saber as melhores peças",
}

user_option = input("""
Bem vindo(a) digite a opção:
1 - Tenho um problema no meu carro, mas não tenho ideia do que seja.
2 - Tenho um problema no meu carro, tenho uma ideia do que seja, mas ainda preciso de ajuda.
3 - Tenho o diagnóstico de um mecânico, mas gostaria de uma segunda opinião.
4 - Já sei do que preciso e necessito de ajuda para achar as peças para o meu carro.
""")

while int(user_option) < 1 or int(user_option) > 4:
  user_option = input("""
    Você digitou uma opção inválida, por favor escolha uma das seguintes opções:
    1 - Tenho um problema no meu carro, mas não tenho ideia do que seja.
    2 - Tenho um problema no meu carro, mas sei parcialmente o que é.
    3 - Tenho um diagnóstico de um mecânico, mas gostaria de uma segunda opinião.
    4 - Já sei do que preciso e necessito de ajuda para achar as peças para o meu carro.
    """)

while continue_chat and num_of_interactions <= 5:
  user_prompt = initial_user_prompt.get(user_option)
  if num_of_interactions == 0:
    response = chat.send_message(system_instruction)
    response = chat.send_message(user_prompt)
  else:
    new_prompt = input("Usuário: ")
    response = chat.send_message(new_prompt)
    print("\n")
    if "fim" == new_prompt.lower():
      continue_chat = False
  num_of_interactions += 1
  print("Mecânico Especialista: ", response.text, "\n")
